In [1]:
import pandas as pd 
import tiingo, datetime
import warnings

warnings.filterwarnings('ignore')

def connect_to_tiingo():
    '''This function establishes a connection to the Tiingo API to donwload historical stock data.'''
    API = 'cb927ca36374f6d4f3455280cb187e4a59fb68f3'
    config = {}
    config['session'] = True
    config['api_key'] = API
    client = tiingo.TiingoClient(config)
    return client

def get_stock_data(ticker, tiingo_conn, startDate=datetime.datetime.now().date() - datetime.timedelta(days=365), endDate=datetime.datetime.now().date(), fmt='json', frequency='daily'):
    '''This function downloads the specified stock data. The function downloads date, close price and volume on the day
    of. The start and end date of time range are preset at January 1, 2018 and November 2, 2018 respectively, with
    daily frequency. The data is returned in json format.
    '''
    stock_data = tiingo_conn.get_ticker_price(ticker, startDate, endDate, fmt, frequency)
    stock_data = pd.DataFrame(stock_data)
    stock_data['date'] = pd.to_datetime(stock_data['date'],format='%Y-%m-%dT%H:%M:%S.%fz')
    stock_data = stock_data.reset_index().set_index(stock_data['date'])
    stock_data['ticker'] = ticker
    return stock_data[['date', 'low', 'high','close', 'volume', 'ticker']]

def macd(df):
    '''This function builds MACD and the 9 day Signal line. 
    
    Parameters:
    -------------
    plot: set to "no" by default. If "yes", outputs a graph of MACD and Signal line
    
    Returns:
    -------------
    DataFrame of MACD and Signal Line    
    '''
    
    df['ema_price_12'] = df['close'].ewm(span = 12, adjust = True, ignore_na=True).mean()
    df['ema_price_26'] = df['close'].ewm(span = 26, adjust = True, ignore_na=True).mean()
    df['macd'] = df['ema_price_12'] - df['ema_price_26']
    df['signal_line'] = df['macd'].ewm(span = 9, adjust = True, ignore_na=True).mean()
    df['macd_diff_signal'] = df['macd'] - df['signal_line']
    
    return df[['date', 'low', 'high','close', 'volume','macd', 'signal_line','macd_diff_signal', 'ticker']]

def rsi(df):
    '''This function builds Relative Strength Indicator. 
    
    Returns:
    -------------
    DataFrame of stock prices and RSI    
    '''
    delta = df['close'].diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    roll_up2 = pd.rolling_mean(up, 14)
    roll_down2 = pd.rolling_mean(down.abs(), 14)
    rsi_raw = roll_up2/roll_down2
    rsi = 100.0 - (100.0 / (1.0 + rsi_raw))
    df['rsi'] = rsi
    return df[['date', 'low', 'high','close', 'volume','macd', 'signal_line', 'macd_diff_signal','rsi', 'ticker']]

def chaikin_oscillator(data, periods_short=3, periods_long=10, high_col='high',
                       low_col='low', close_col='close', vol_col='volume'):
    ac = pd.Series([])
    val_last = 0
    
    for index, row in data.iterrows():
        if row[high_col] != row[low_col]:
            val = val_last + ((row[close_col] - row[low_col]) - (row[high_col] - row[close_col])) / (row[high_col] - row[low_col]) * row[vol_col]
        else:
            val = val_last
        ac.set_value(index, val)
    val_last = val

    ema_long = ac.ewm(ignore_na=False, min_periods=0, com=periods_long, adjust=True).mean()
    ema_short = ac.ewm(ignore_na=False, min_periods=0, com=periods_short, adjust=True).mean()
    data['ch_osc'] = ema_short - ema_long

    return data

def volume(row):
    if row['mean_month_vol'] < 100000:
        return 'low volume'
    else:
        return 'high volume'  
    
def main():
    connection = connect_to_tiingo()

    stacked = pd.DataFrame(columns=['date','close','rsi','macd_diff_signal','macd','signal_line', 'ticker'])
    for i in pd.read_excel('/Users/Mahmud/Desktop/stock_market_tracker/test_track.xlsx')['Ticker']:
        try:
            df = get_stock_data(i, connection)
            df = macd(df)
            df = rsi(df)
            df = chaikin_oscillator(df)
            stacked = stacked.append(df)
            stacked['date'] = stacked.index
            print('Retrieving ', i)
 
        except:
            print('Unable to retrieve ', str(i))
            
    
    stacked.to_csv('/Users/Mahmud/Desktop/stock_market_tracker/test.csv', index=False)
    
    return stacked

if __name__ == "__main__":
    main()


Unable to retrieve  MDCA
Unable to retrieve  GIL
Unable to retrieve  LULU
Unable to retrieve  SOLO
Unable to retrieve  MGA
Unable to retrieve  F
Unable to retrieve  GM
Unable to retrieve  TSLA
Unable to retrieve  COT
Unable to retrieve  YVR
Unable to retrieve  TAP
Unable to retrieve  APTO
Unable to retrieve  NYMX
Unable to retrieve  BAM
Unable to retrieve  PCOM
Unable to retrieve  RBA
Unable to retrieve  RY
Unable to retrieve  TD
Unable to retrieve  WFC
Unable to retrieve  BMO
Unable to retrieve  BMLP
Unable to retrieve  CM
Unable to retrieve  MFCB
Unable to retrieve  RY
Unable to retrieve  TD
Unable to retrieve  BB
Unable to retrieve  CLDR
Unable to retrieve  CPAH
Unable to retrieve  SHOP
Unable to retrieve  DSGX
Unable to retrieve  ALYA
Unable to retrieve  TSG
Unable to retrieve  PM
Unable to retrieve  WMT
Unable to retrieve  BGI
Unable to retrieve  OTEX
Unable to retrieve  ANY
Unable to retrieve  TCX
Unable to retrieve  TAC
Unable to retrieve  CLS
Unable to retrieve  EXFO
Unable to 

ERROR:root:b'{"detail":"Error: Ticker \'TICKER\' not found"}'


Unable to retrieve  DOOO
Unable to retrieve  ticker
Unable to retrieve  IMO
Unable to retrieve  SU
Unable to retrieve  MFC
Unable to retrieve  SLF
Unable to retrieve  BNS
Unable to retrieve  FSI
Unable to retrieve  HYGS
Unable to retrieve  LOOP
Unable to retrieve  MEOH
Unable to retrieve  ACST
Unable to retrieve  AEZS
Unable to retrieve  AQXP
Unable to retrieve  ABUS
Unable to retrieve  AUPH
Unable to retrieve  BHC
Unable to retrieve  CMTA
Unable to retrieve  CORV
Unable to retrieve  DMPI
Unable to retrieve  EPIX
Unable to retrieve  IMV
Unable to retrieve  IPCI
Unable to retrieve  NEPT
Unable to retrieve  NVLN
Unable to retrieve  ONCY
Unable to retrieve  SRRA
Unable to retrieve  TRIL
Unable to retrieve  XENE
Unable to retrieve  ZYME
Unable to retrieve  TEVA
Unable to retrieve  NFLX
Unable to retrieve  SPOT
Unable to retrieve  NVCN
Unable to retrieve  LMT
Unable to retrieve  STN
Unable to retrieve  AKG
Unable to retrieve  GSV
Unable to retrieve  TECK
Unable to retrieve  QTRH
Unable to r

ERROR:root:b'{"detail":"Error: Ticker \'GRP.U\' not found"}'


Unable to retrieve  FSV
Unable to retrieve  GRP.U
Unable to retrieve  QSR
Unable to retrieve  SNAP
Unable to retrieve  CGC
Unable to retrieve  CRON
Unable to retrieve  TLRY
Unable to retrieve  AMD
Unable to retrieve  AMZN
Unable to retrieve  AAPL
Unable to retrieve  BA
Unable to retrieve  EFX
Unable to retrieve  SHOP
Unable to retrieve  BCE
Unable to retrieve  TU
Unable to retrieve  RCI
Unable to retrieve  SJR
Unable to retrieve  ACB


In [97]:
# Pending Updates

#1 add functions for MACD, RSI, CH_OSC that classify them as BUY, HOLD OR SELL
#2 move ch_osc to the PA website
#3 write the conditions below into a function, use apply to create a column and label it buy/sell/hold, see example below
#4 modularize the code into dependencies and make it callable from a main.py function
#5 rename file names

' def set_color(row):\n    if row["Set"] == "Z":\n        return "red"\n    elif row["Type"] == "C":\n        return "blue"\n    else:\n        return "green"\n\ndf = df.assign(color=df.apply(set_color, axis=1))\n\nprint(df)'

In [49]:
#1 below 0:
    #1.1 above global min: maybe try using local min, 
        #1.1a declined last day ## LOOK AT PERCENTAGE DIFFERENCE OF THE DECLINE VS INCLINE TO IDENTIFY THE ACCELERATION 
                            ## OF THE MOMENTUM
            # 'buy', alert
        #1.1b increased last day
            # 'hold', no alert
        
    #1.2 below global min:
        #1.2a declined last day
            # 'buy', alert
        #1.2b increased last day
            # 'sell', no alert

#2 above 0: revert the above to calculations around global max
    # 


In [105]:
#4
df = pd.read_csv('/Users/Mahmud/Desktop/stock_market_tracker/test.csv')
df = df[(df['ticker']=='BCE') & (df['date'] <= '2019-05-07')]

# find the day over day change in macd_diff_signal, first difference
df['delta'] = df['macd_diff_signal'].diff()
df.head(5)

# find the second difference, to measure % increase/decrease in day over day delta
#.....

# find global min
global_min = df['macd_diff_signal'].min()

# above vs below the global min
df['gl_min_ab_be'] = np.where(df['macd_diff_signal']<=global_min,'below','above')


#print('------ Part 1.1 ------')
#1.1a: macd_diff below 0, above global min, declined last day (BUY / PAY ATTENTION)
df_above_min_dec = df[(df['gl_min_ab_be'] == 'above') & (df['delta'] < 0) & (df['rsi'] < 30) 
                      & (df['macd_diff_signal'] < 0)]

#1.1b: macd_diff below 0, above global min, increased last day (SELL / HOLD)
df_above_min_inc = df[(df['gl_min_ab_be'] == 'above') & (df['delta'] > 0) & (df['rsi'] < 30)
                     & (df['macd_diff_signal'] < 0)]


#print('------ Part 1.2 ------')
#1.2a: macd_diff below 0, below global min, declined last day (BUY / PAY ATTENTION)
df_below_min_dec = df[(df['gl_min_ab_be'] == 'below') & (df['delta'] < 0) & (df['rsi'] < 30)
                     & (df['macd_diff_signal'] < 0)]

#1.2b: macd_diff below 0, below global min, increased last day (SELL / HOLD)
df_below_min_inc = df[(df['gl_min_ab_be'] == 'below') & (df['delta'] > 0) & (df['rsi'] < 30)
                     & (df['macd_diff_signal'] < 0)]


,ch_osc,close,date,high,low,macd,macd_diff_signal,rsi,signal_line,ticker,volume,delta,gl_min_ab_be
45638,-402169.859476,39.32,2018-12-24,39.89,39.26,-0.344525,-0.400622,13.211382,0.056098,BCE,959392.0,-0.059784,below
